In [95]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
import time

def get_headers():
    """Make headers to access url"""
    headers="""accept: application/json
    cid: undefined
    content-type: application/json
    Referer: https://www.walmart.com/m/electronics?page=2&ps=60
    sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"
    sec-ch-ua-mobile: ?0
    User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36
    useragent: undefined"""
    headers = dict([i.split(': ') for i in headers.split('\n')])
    return headers

def get_data():
    """Use headers to access URL->loop through url pages->
    resp/json to df-> append each df to df_walmart"""
    df_walmart= pd.DataFrame()
    url1 ="""https://www.walmart.com/cp/api/get-deals-list?prg=desktop&dealsId=electronics&ps=60&page="""
    url2 ="""&shelf_id=5788839&shelfType=manual"""
    for i in range(1,5):
        url = url1+f'{i}'+url2
        response = r.get(url, headers = headers)
        data=response.json()
        data= pd.json_normalize(data['items'])
        df_walmart = df_walmart.append(data)
        time.sleep(5) 
    return df_walmart
    
def save_copy(df_walmart):  
    """Save org df and make a copy to clean"""
    df_walmart.reset_index(drop=True, inplace=True)
    df_walmart.to_csv('.\output\org_walmart.csv', index=False)
    df_copy = df_walmart.copy()
    return df_copy

def clean_df(df_copy):
    """clean copy of df to make clean df"""
    df_copy.drop(columns = ['usItemId', 'shippingPassEligible', 'wfsEnabled', 'pickupDiscountEligible', 'preOrderAvailable', 'virtualPack', 'premiumBrand', 'is_limited_qty', 'isLimitedQuantity','visibleSwatches','primaryOffer.offerId', 'fulfillment.isS2H', 'fulfillment.isS2S','fulfillment.isSOI', 'fulfillment.isPUT', 'fulfillment.s2SDisplayFlags','ppu.unit', 'ppu.amount', 'ppu.currencyCode','mediaLanguage', 'sellerBadge','variants.variantMeta', 'variants.variantData', 'isbn', 'mediaFormat','primaryOffer.minPrice', 'primaryOffer.maxPrice','primaryOffer.showMinMaxPrice', 'preOrderAvailableDate', 'launchDate','submapType', 'bundleType', 'primaryOffer.installmentFrequency','primaryOffer.durationWithInterest.duration','primaryOffer.durationWithInterest.durationUOM','primaryOffer.durationWithInterest.interest','primaryOffer.durationWithInterest.interestUOM','inventory.displayFlags', 'fulfillment.thresholdAmount','fulfillment.thresholdCurrencyCode', 'mediaRating','primaryOffer.unitPriceDisplayCondition', 'consoleKstem','minItemCountPerOrder', 'mediaRunningTime','geoItemClassification', 'giftingEligible', 'pcsType','wmtgPricePerUnitQuantity',  'isHeartable','countPerPack', 'isWMPlusShipping', 'blitzItem', 'marketPlaceItem','twoDayShippingEligible', 'specialOfferBadge', 'specialOfferText','specialOfferLink', 'sellerId', 'sellerName', 'enableAddToCart','canAddToCart', 'cta', 'showPriceAsAvailable', 'itemClassId', 'primaryOffer.currencyCode', 'primaryOffer.showWasPrice', 'inventory.availableOnline', 'productPageUrl', 'fulfillment.s2HDisplayFlags','productType', 'imageProps.src', 'imageProps.srcSet', 'seeAllLink', 'department','seeAllName'], inplace=True)
    df_copy = df_copy.loc[:, ['productId', 'primaryOffer.savingsAmount', 'primaryOffer.offerPrice', 'primaryOffer.listPrice', 'title', 'description', 'productCategory', 'customerRating', 'numReviews', 'brand', 'initialQuantity', 'quantity','standardUpc','imageUrl']]
    df_copy['description'] = df_copy['description'].str.replace('(<li>)', '').str.replace('(</li>)', '').str.replace('<sup>','').str.replace('</sup>','').str.replace('<b>','').str.replace('</b>','').str.replace('<div>','').str.replace('</div>','').str.replace('<p>','').str.replace('</p>','').str.replace('p class="msonormal">', '')
    df_copy['description']= df_copy['description'].str.replace('(<.*>)', '')
    df_copy['description']=df_copy['description'].fillna(value='')
    cols_rename = {'primaryOffer.savingsAmount': 'Savings$' ,'primaryOffer.offerPrice': 'Sale_price$', 'primaryOffer.listPrice': 'Orig_price$', 'title':'Item'}
    df_copy.rename(columns = cols_rename, inplace=True)
    df_copy.dropna(inplace=True)
    df = df_copy
    return df

def save_df(df):
    """save final df"""
    df.to_csv('.\output\walmart.csv', index=False)
    
if __name__ == '__main__':
    df_walmart = get_data()
    df_copy = save_copy(df_walmart)
    df = clean_df(df_copy)
    save_df(df)